In [ ]:
import ...

In [ ]:
DATASET_FOLDER = '/tmp/testset'

In [ ]:
... code ...

In [ ]:
... code ...

In [ ]:
... code ...

In [ ]:
!pip install pycocotools

# Generate output json

In [ ]:
# PSEUDO CODE FOR A GENERIC MODEL!!!
results = []

for image_data in dataloader:
    inputs = image_data['inputs']
    classes, scores, boxes, masks = model(inputs)

    classes = classes.cpu().numpy()
    scores = scores.cpu().numpy()
    boxes = boxes.view(-1, 4).cpu().numpy()
    masks = masks.view(-1, h, w).cpu().numpy()
    for i in range(masks.shape[0]):
        image_id = image_data[i]['image_id']

        # box defined as (x1, y1, x2, y2)
        bbox = boxes[i]
        bbox = [bbox[0], bbox[1], bbox[2]-bbox[0], bbox[3]-bbox[1]]
        # Round to the nearest 10th to avoid huge file sizes, as COCO suggests
        bbox = [round(float(x)*10)/10 for x in bbox]

        rle = pycocotools.mask.encode(np.asfortranarray(masks[i].astype(np.uint8)))
        rle['counts'] = rle['counts'].decode('ascii') # json.dump doesn't like bytes strings

        results.append({
                'image_id': int(image_id),
                'category_id': get_coco_category(int(classes[i])),
                'bbox': bbox,
                'segmentation': rle,
                'score': float(scores[i])
            })
    
# save file
with open('pred.json', 'w') as fp:
    json.dump(results, fp)

# Compute metrics

In [1]:
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

In [2]:
test = '/corso_analisi_immagini_2023_2024/lezioni/test.json'

gt = COCO(test)

gt.cats

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


{0: {'supercategory': 'cable', 'id': 0, 'name': 'cable'},
 1: {'supercategory': 'tower_lattice', 'id': 1, 'name': 'tower_lattice'},
 2: {'supercategory': 'tower_tucohy', 'id': 2, 'name': 'tower_tucohy'},
 3: {'supercategory': 'tower_wooden', 'id': 3, 'name': 'tower_wooden'}}

In [3]:
gt = COCO(test)
detections = gt.loadRes('/corso_analisi_immagini_2023_2024/lezioni/bbox_detections.json')

detections.cats

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.14s)
creating index...
index created!


{0: {'supercategory': 'cable', 'id': 0, 'name': 'cable'},
 1: {'supercategory': 'tower_lattice', 'id': 1, 'name': 'tower_lattice'},
 2: {'supercategory': 'tower_tucohy', 'id': 2, 'name': 'tower_tucohy'},
 3: {'supercategory': 'tower_wooden', 'id': 3, 'name': 'tower_wooden'}}

In [4]:
# running evaluation
annType = 'bbox'
cocoEval = COCOeval(gt, detections, annType)
#cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=3.23s).
Accumulating evaluation results...
DONE (t=0.40s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.239
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.460
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.238
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.177
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.187
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.318
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.221
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.319
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.333
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.224
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100

In [5]:
# running evaluation
annType = 'segm'
detections = gt.loadRes('/corso_analisi_immagini_2023_2024/lezioni/mask_detections.json')
cocoEval = COCOeval(gt, detections, annType)
#cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Loading and preparing results...
DONE (t=0.47s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=3.70s).
Accumulating evaluation results...
DONE (t=0.44s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.189
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.344
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.174
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.027
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.110
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.368
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.263
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.265
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=10

In [8]:

with open('/corso_analisi_immagini_2023_2024/lezioni/bbox_detections.json') as fp_src:
    data = json.load(fp_src)
    
print(type(data), len(data))
data[0]

<class 'list'> 30269


{'image_id': 0,
 'category_id': 0,
 'bbox': [291.0, 40.0, 86.0, 653.0],
 'score': 0.3679635524749756}

In [7]:

with open('/corso_analisi_immagini_2023_2024/lezioni/mask_detections.json') as fp_src:
    data = json.load(fp_src)
    
print(type(data), len(data))
data[0]

<class 'list'> 30269


{'image_id': 0,
 'category_id': 0,
 'segmentation': {'size': [700, 700],
  'counts': 'kUj68_e08i\\1OScN4W]10ebN4oR2CeZO6`bN3j\\1OQ]N@lFa0_l19_bN2n\\1E\\cN0hV1FPZNe0a8[O`Gf0Qe1DQ[N7V8HlG7Se1NlZN;h7EYH;oU1ZOlQOa0WH0c7M`H0nn0BkQOLe6f0dH4R7LoH3^7_OQI4Z73^H4n64lH0Q7MRI0Sd1FjjNGe\\1>WcNESbi6'},
 'score': 0.3679635524749756}